In [1]:
#importing modules
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


from langdetect import detect_langs
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer



In [2]:
# #twitter api keys
# auth = tweepy.OAuth1UserHandler(
#    )

# api = tweepy.API(auth, wait_on_rate_limit=True)


In [3]:
# search_words = "#wakandaforever OR #WakandaForever OR #blackpanther OR #WakandaForeverID OR #BlackPantherWakandaForever OR #BlackPanther OR #BlackPanther2"
# search_query = search_words + " -filter:retweets AND -filter:replies"


# limit = 100000
# tweets = tweepy.Cursor(api.search_tweets, q=search_query, tweet_mode='extended').items(limit)

# data=[]
# for tweet in tweets:
#     data.append([tweet.id,tweet.user.screen_name, tweet.created_at, 
#                   tweet.full_text,tweet.user.location, tweet.retweet_count,tweet.favorite_count,
#                    tweet.user.followers_count, tweet.user.friends_count, tweet.user.verified, tweet.source])

In [4]:
# columns =['id','username','time_of_tweet', 'tweet', 'location', 
#          'retweets', 'likes', 'followers', 'following', 'verified', 'tweet_source']


# #convert to dataframe and import into csv
# bp_tweets=pd.DataFrame(data, columns=columns)
# bp_tweets.to_csv("BlackPanther-24-11.csv")

In [5]:
#save scrapped data to csv 
tweets = pd.read_csv('BlackPanther-24-11.csv')
tweets.sample(10)

,Unnamed: 0,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source
17914,17914,1593834178763640832,LuWoodridge,2022-11-19 05:10:54+00:00,#BlackPantherWakandaForever commanded respect....,NaN,0,0,10,44,False,Twitter Web App
24656,24656,1593285943104049153,TRACE_Toca,2022-11-17 16:52:25+00:00,"“Precisamos começar a nos sentir orgulhosos, b...",johannesburg / Paris,1,4,5268,253,False,Twitter for iPhone
25459,25459,1593238545401282560,Eduardo96311281,2022-11-17 13:44:04+00:00,APENAS ASSISTA E AJUDE QUE DEUS MULTIPLIQUE 10...,São Paulo/Brasil,1,0,17,48,False,Twitter Web App
13377,13377,1594171591767760896,itsBGbitch,2022-11-20 03:31:40+00:00,It's a black panther Jr #WakandaForever #Black...,Chattanooga,0,0,1305,1606,False,Twitter for Android
30457,30457,1592926322606895104,TheJaykishan1,2022-11-16 17:03:25+00:00,Fabulous\n\nOne of the best #Marvel flick in r...,"Bengaluru, India",0,0,541,54,False,Twitter for Android
23077,23077,1593384427400400896,yarilbooks,2022-11-17 23:23:45+00:00,Ame sentirme representado en #wakandaforever L...,"Benito Juárez, Quintana Roo",0,2,51,300,False,Twitter for Android
16410,16410,1593993083124912129,TheNoviceOAO,2022-11-19 15:42:20+00:00,"loved #WakandaForever, but agree, subtext of i...",London,2,2,28,50,False,Twitter Web App
20070,20070,1593673569489149953,culi5_,2022-11-18 18:32:42+00:00,Here super early enjoying my chicken tenddies ...,SF Bay Area,0,0,416,486,False,Twitter for iPhone
10717,10717,1594399029496250368,goldloki,2022-11-20 18:35:25+00:00,thx @MarvelStudios for spoiling who the next b...,tønsberg - 20+,0,1,1130,538,False,Twitter for iPhone
21821,21821,1593493535223054337,telugu_nidhhi,2022-11-18 06:37:19+00:00,💘💘💘 @AgerwalNidhhi 💖💖💖\n#NidhhiAgerwal #Nidhhi...,NaN,0,2,44,1,False,Twitter for Android


In [6]:
#column names and datatype
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35054 entries, 0 to 35053
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     35054 non-null  int64 
 1   id             35054 non-null  int64 
 2   username       35054 non-null  object
 3   time_of_tweet  35054 non-null  object
 4   tweet          35054 non-null  object
 5   location       24584 non-null  object
 6   retweets       35054 non-null  int64 
 7   likes          35054 non-null  int64 
 8   followers      35054 non-null  int64 
 9   following      35054 non-null  int64 
 10  verified       35054 non-null  bool  
 11  tweet_source   35054 non-null  object
dtypes: bool(1), int64(6), object(5)
memory usage: 3.0+ MB


In [7]:
#statistical description of the dataset
tweets.describe()

,Unnamed: 0,id,retweets,likes,followers,following
count,35054.000000,3.505400e+04,35054.000000,35054.000000,3.505400e+04,35054.000000
mean,17526.500000,1.593928e+18,4.522194,37.315513,3.897041e+04,1241.136247
std,10119.362505,8.477788e+14,63.704964,630.220989,5.791979e+05,6060.701871
min,0.000000,1.592672e+18,0.000000,0.000000,0.000000e+00,0.000000
25%,8763.250000,1.593155e+18,0.000000,0.000000,1.080000e+02,149.000000
50%,17526.500000,1.593876e+18,0.000000,1.000000,4.150000e+02,420.000000
75%,26289.750000,1.594521e+18,0.000000,3.000000,1.547000e+03,1047.000000
max,35053.000000,1.595755e+18,4870.000000,51334.000000,4.083612e+07,301618.000000


In [8]:
#drop unnamed column
tweets.drop(columns='Unnamed: 0', inplace = True)

#fill in missing location with 'None'
tweets['location'].fillna('None', inplace=True)

#change the datatype of tweet_time to datetime 
tweets['time_of_tweet'] = tweets['time_of_tweet'].astype('datetime64')

In [9]:
def get_hash(tweet):
    '''This function collect all the hashtags in the tweet column'''
    tweet = re.findall(r'\#\w+', tweet)
    return " ".join(tweet)

In [10]:
#applying the get_hash function
tweets['hashtags'] = tweets['tweet'].apply(get_hash)


In [11]:
#count the occurrence of each hashtags in the hashtags columns
hashtags = pd.Series(" ".join(tweets['hashtags']).split()).value_counts()
#hashtags.to_csv('hashtag_freq.csv')    ##save the output as a csv
hashtags[:10]

#WakandaForever                20990
#BlackPanther                   9316
#BlackPantherWakandaForever     5887
#wakandaforever                 1997
#Namor                          1746
#blackpanther                   1297
#Marvel                         1211
#BlackPanther2                   998
#WakandaForeverID                978
#MarvelStudios                   808
dtype: int64

In [12]:
def get_languages(tweet):
    '''This function detects the language in each 
    tweet and returns it in ISO 639-1 codes'''
    try:
        tweet = detect_langs(tweet)
    except: 
        tweet = "nolang"
    tweet = str(tweet).split(':')[0][1:]
    return tweet

In [13]:
#applying the get_languages function
tweets['language'] = tweets['tweet'].apply(get_languages)

In [14]:
#substituting the short form language names for the long names
lang_code = pd.read_csv('language_codes.csv')
dict_lang_code = dict(zip(lang_code['alpha2'], lang_code['English']))
tweets['language'] = tweets['language'].replace(dict_lang_code)

In [15]:
#checking for changes
tweets.language.value_counts()[:10]

English       24202
Spanish        3325
Danish         1502
Portuguese     1462
French          772
Indonesian      508
Japanese        473
Thai            468
Norwegian       368
Italian         335
Name: language, dtype: int64

In [16]:
def get_links(tweets):
    '''This function iterate over the tweets and return 
    'Yes' is there's a link and 'No' if there's no link '''
    link = re.findall(r'((http|ftp|https):\/\/)(([\w.-]*)\.([\w]*))', tweets)
    if len(link) < 1:
        return 'No'
    else: return 'Yes'


In [17]:
#applying the get_links function
tweets['contains_media'] = tweets['tweet'].apply(get_links)

In [18]:
#checking for changes
tweets.contains_media.value_counts()

Yes    20872
No     14182
Name: contains_media, dtype: int64

In [19]:
cast = ['namor', 'shuri', 'ironheart', 'ramonda', 'namora', 
        'okoye', 'aneka', 'nakia', 'mbaku', 'attuma', 'everett', ]


def get_cast(tweets):
        '''This function extracts names of the cast into a new column'''      
        tweets = tweets.lower()
        tokens = word_tokenize(tweets)
        cast_names = [name for name in tokens if name in cast]
        cast_names_title = [name.title() for name in cast_names]
        return " ".join(cast_names_title)

In [20]:
#applying the get_cast function
tweets['cast_name'] = tweets['tweet'].apply(get_cast)

In [21]:
#count the occurrence of each name in the cast_name columns
names = pd.Series(" ".join(tweets['cast_name']).split()).value_counts()
# names.to_csv('names_freq.csv') ##save the generated dataframe
names[:10]

Namor        3953
Shuri        1985
Okoye         348
Namora        295
Ironheart     288
Nakia         220
Ramonda       216
Mbaku         202
Attuma        123
Aneka          53
dtype: int64

In [22]:
#replace the '\n' characters with space
tweets['tweet'] = tweets['tweet'].replace('\n', ' ', regex=True)

In [23]:
#get the sentiment scores using Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()
tweets['sentiment_scores'] = tweets['tweet'].apply(sia.polarity_scores)
tweets['sentiment_scores'] = tweets['sentiment_scores'].apply(lambda x: x['compound'])

In [24]:
def get_sentiment(tweet):
    '''This functions returns Positive, Negative or 
    Neutral depending on the state of the sentiment score'''
    if tweet > 0:
        return "Positive"
    elif tweet == 0:
        return "Neutral"
    else: return "Negative"

In [25]:
#applying the get_sentiment function
tweets['sentiment'] = tweets['sentiment_scores'].apply(get_sentiment)

In [26]:
#checking for changes
tweets['sentiment'].value_counts()

Neutral     17445
Positive    12441
Negative     5168
Name: sentiment, dtype: int64

In [27]:
def get_country(inc_c, case, cor_c):
    '''This function substitutes the current location for the country name'''
    tweets.loc[tweets['c_location'].str.contains(inc_c, case=case), 'c_location'] = cor_c


In [29]:
#duplicate the location column to another column 
tweets['c_location'] = tweets['location']

#substitute incorrect spelling for correct spelling
dicts = {'brazil':'Brazil', 'brasil':'Brazil', 'nigeria':'Nigeria', 'méxico': 'Mexico', 'france':'France', 
        'india':'India', 'london':'United Kingdom', 'UK':'United Kingdom', 'england':'United Kingdom',
        'Bonaire':'Bonaire Islands', 'España':'Spain', 'thailand':'Thailand', 'Ohio':'United States',
        'South Africa':'South Africa'}
for key, val in dicts.items():
    get_country(key, False, val)

#substitute state names in the US
list_us = ['USA', 'KS', 'FL', 'CA', 'GA', 'AZ', 'CO','IL', 'TX', 'PA', 'DC', 'TN', 'NY', 'WA', 'NYC', 'NC', 'NV', 'OH', 'LA', 'MA', 'Los Angeles' 'new york','New York', 'Los Angeles']
for states in list_us:
    get_country(states, True, 'United States')

#minor formatting
tweets['c_location'].replace(['None', 'she/her'], np.nan, inplace=True)
tweets['c_location'].value_counts()[:10]

United States     5768
United Kingdom    1123
Brazil             666
Mexico             582
India              572
France             311
Spain              308
Nigeria            193
South Africa       190
Thailand           104
Name: c_location, dtype: int64

In [30]:
#frequency of tweet_source
tweets['tweet_source'].value_counts()[:10]

Twitter for iPhone      15298
Twitter for Android     11002
Twitter Web App          5560
TweetDeck                 575
Twitter for iPad          331
Instagram                 289
Hootsuite Inc.            218
Buffer                    174
IFTTT                     141
Twitter Media Studio      139
Name: tweet_source, dtype: int64

In [31]:
#frequency of verified or not
tweets['verified'].value_counts()

False    33570
True      1484
Name: verified, dtype: int64

In [32]:
#final sampling of the dataset
tweets.sample(20)

,id,username,time_of_tweet,tweet,location,retweets,likes,followers,following,verified,tweet_source,hashtags,language,contains_media,cast_name,sentiment_scores,sentiment,c_location
18707,1593771055809699840,fangirlish,2022-11-19 01:00:05,#wakandaforever is setting the new age of Marv...,all over the world,0,0,19544,134,False,CoSchedule,#wakandaforever #blackpanther #ironheart,English,Yes,Ironheart,0.4215,Positive,all over the world
14048,1594130553476771840,Vifyo_,2022-11-20 00:48:36,#WakandaForever,Mars,0,3,46,421,False,Twitter for iPhone,#WakandaForever,Danish,No,,0.0000,Neutral,Mars
22364,1593439406169489408,MeshalAS87,2022-11-18 03:02:13,[[[[[[💸$400💸400💸000💸]]]]]] #BlackPant...,Kingdom of Saudi Arabia,4,15,1247,3536,False,Twitter for iPhone,#BlackPanther #Namor #WakandaForever #atAMC #A...,English,Yes,Namor,0.0000,Neutral,Kingdom of Saudi Arabia
25724,1593217348382916609,OopsOhMai,2022-11-17 12:19:51,Watching #WakandaForever tonight in Dubai Mall...,"Dubai, UAE",0,5,359,344,False,Twitter for iPhone,#WakandaForever,Tagalog,No,,0.5983,Positive,"Dubai, UAE"
14542,1594103156685807621,MatheusVFig,2022-11-19 22:59:44,#WakandaForever,None,0,0,40,212,False,Twitter for Android,#WakandaForever,Danish,No,,0.0000,Neutral,NaN
20555,1593638915449184256,DCBlackGeeks,2022-11-18 16:15:00,As we get ready for @theblackpanther: Wakanda ...,None,0,0,990,1385,False,Twitter Web App,#BlackPantherSoLit #WakandaForever #BlackPanther,English,Yes,,0.7717,Positive,NaN
10542,1594406352452845568,ChillerPop,2022-11-20 19:04:31,It's important to note that #WakandaForever is...,None,0,3,3386,4981,False,Twitter for Android,#WakandaForever,English,No,,0.4939,Positive,NaN
31060,1592898950994227203,CyB3rPh0b1a,2022-11-16 15:14:39,La historia jamás contada por Marvel #BlackPan...,España,0,0,80,83,False,Twitter for Android,#BlackPanther,Spanish,Yes,,0.4215,Positive,Spain
26992,1593096468172525568,nadine_feiler,2022-11-17 04:19:30,#WakandaForever So now i am to like a totalit...,None,0,2,1024,98,False,Twitter Web App,#WakandaForever,English,No,,-0.4512,Negative,NaN
9519,1594466612437917696,erudite1906,2022-11-20 23:03:58,She shared more than just her training. #nowik...,"Los Angeles, CA",0,0,884,2737,False,Twitter for Android,#nowiknow #mercy #WakandaForever,English,No,,0.3400,Positive,United States


In [ ]:
#export to csv
tweets.to_csv('BlackPanther_Cleaned.csv')